In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import linalg

In [ ]:
def beta_fn(n, sigma):
    return -1 * (1 + sigma) / n / 2

def de1(n, rho, sigma, beta):
    matrix = np.eye(n)
    matrix[:rho+1, sigma] -= 1
    matrix[rho+1:, sigma] -= (1 + beta)
    return matrix
        
def de2(n, rho, beta):
    matrix = np.eye(n)
    matrix[:rho+1, rho] -= 1
    matrix[rho+1:, rho] -= (1 + beta)
    return matrix
        
def de3(n, rho, beta):
    matrix = np.eye(n)
    matrix[:rho+1, n-1] -= 1 / (1 + beta)
    matrix[rho+1:, n-1] -= 1
    return matrix
        
def de4(n):
    matrix = np.eye(n)
    idx = [-1] + list(range(n-1))
    matrix = matrix[idx]
    return matrix


In [ ]:
def compute_dF(n, sigma, rho, sfirst):
    if sfirst:
        return (
            de4(n) @
            de3(n, rho-1, beta_fn(n, sigma - 1)) @
            de2(n, rho, beta_fn(n, sigma - 1)) @
            de1(n, rho, sigma, beta_fn(n, sigma))
        )[1:, 1:]
    else:
        return (
            de4(n) @
            de3(n, rho-1, beta_fn(n, sigma - 1)) @
            de1(n, rho-1, sigma, beta_fn(n, sigma)) @
            de2(n, rho, beta_fn(n, sigma))
        )[1:, 1:]

In [ ]:
def determine_stability(n, sigma, rho, sfirst):
    dF = compute_dF(n, sigma, rho, sfirst)
    max_eig_norm = np.abs(linalg.eigvals(dF)).max()
    epsilon = 1e-6
    if max_eig_norm > 1 + epsilon:
        return 'unstable'
    elif max_eig_norm < 1 - epsilon:
        return 'stable'
    else:
        return 'neutral'

In [ ]:
n = 9
plt.fill([0, 0, n], [0, n, n], fill=None)
color_map = {
    'stable': 'blue',
    'unstable': 'red',
    'neutral': 'white'
}
for sigma in range(n):
    for rho in range(sigma, n):
        stability = determine_stability(n, sigma, rho, sfirst=True)
        color = color_map[stability]
        plt.fill(
            [sigma, sigma, sigma+1], 
            [rho, rho+1, rho+1], 
            color, 
            label=1
        )
        if rho > sigma:
            stability = determine_stability(n, sigma, rho, sfirst=False)
            color = color_map[stability]
            plt.fill(
                [sigma, sigma+1, sigma+1], 
                [rho, rho, rho+1], 
                color
            )